In [1]:
# MacLight – GFLOPs & Time‑Budget Notebook  
# ------------------------------------------------------------
# This script can be executed as a Jupyter notebook (VS Code, JupyterLab)
# or run top‑to‑bottom as a plain Python file.  Cells are delimited by
# `# %%` markers so VS Code automatically recognises them.
#
# ▶ WHAT IT DOES
#   • loads MacLight networks (VAE, PolicyNet, ValueNet)
#   • measures MACs / GFLOPs with `ptflops`
#   • multiplies by call‑counts to give GFLOPs per EPISODE and for the
#     whole 80‑episode training run in the original paper
#   • prints a small timing estimate based on the *current GPU* 
#     (useful to compare laptop vs Colab A100)
# ------------------------------------------------------------

In [2]:
# %% 0  – Install dependencies if needed
%pip -q install ptflops 
%pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
# %% 1  – Imports and repo path
import os, sys, time, math, torch
from pathlib import Path
from ptflops import get_model_complexity_info

#  ↳ if running in Colab, clone the repo automatically
if "google.colab" in sys.modules:
    if not Path("MacLight").exists():
        !git clone -q https://github.com/wissem18/MacLight.git
    repo_root = Path("MacLight")
    os.chdir(repo_root)
else:
    # local execution
    repo_root = Path.cwd()
    assert (repo_root / 'run_Ours.py').exists(), "Run inside the MacLight repo root"

sys.path.append(str(repo_root))

print(f"▶ Using repo at: {repo_root}\n")

▶ Using repo at: /Users/wissem.yousfi/Desktop/Work/Codes/MacLight



In [14]:
# %% 2  – Load network definitions
from net.net import  PolicyNet, ValueNet, VAE, Attention

# Dimensions taken from original config
STATE_DIM   = 33    
LATENT_DIM  = 10     
HIDDEN_DIM  = STATE_DIM * 2  
ACTION_DIM  = 8        

ImportError: cannot import name 'Attention' from 'net.net' (/Users/wissem.yousfi/Desktop/Work/Codes/MacLight/net/net.py)

In [4]:
# %% 3  – Helper to compute GFLOPs

def gflops_and_params(model, inp_shape):
    macs, params = get_model_complexity_info(model, inp_shape,
                                             as_strings=False,
                                             print_per_layer_stat=False)
    return (macs * 2) / 1e9, params / 1e6


In [5]:
# %% 4  – Instantiate models
vae   = VAE(STATE_DIM, LATENT_DIM)
policy = PolicyNet(STATE_DIM, HIDDEN_DIM, ACTION_DIM)
value  = ValueNet(STATE_DIM, HIDDEN_DIM, LATENT_DIM)

NameError: name 'VAE' is not defined

In [15]:
# %% 5  – GFLOPs per forward pass
vae_g, vae_p = gflops_and_params(vae,   (STATE_DIM, 4, 4))
pol_g, pol_p = gflops_and_params(policy, (STATE_DIM ,))
val_g, val_p = gflops_and_params(value,  (STATE_DIM + LATENT_DIM,))

print(f"VAE                : {vae_g:.4f} GFLOPs | {vae_p:.2f} M params")
print(f"PolicyNet          : {pol_g:.4f} GFLOPs | {pol_p:.2f} M params")
print(f"ValueNet           : {val_g:.4f} GFLOPs | {val_p:.2f} M params")

VAE                : 0.0178 GFLOPs | 0.81 M params
PolicyNet          : 0.0000 GFLOPs | 0.01 M params
ValueNet           : 0.0000 GFLOPs | 0.01 M params


In [21]:
# %% 6  – Total GFLOPs per episode & full training run
T, K, E, L = 720, 16, 10, 80
vae_tot    = 3 * vae_g * T
actor_tot  = 3 * pol_g * T * K
critic_tot = 3 * val_g * T * K * (1 + E)

per_episode = vae_tot + actor_tot + critic_tot
print(f"GFLOPs for VAE per episode  : {vae_tot: .2f}")
print(f"GFLOPs per episode   : {per_episode:.2f}")
print(f"GFLOPs for {L} eps : {per_episode*L:.2f} \n")

GFLOPs for VAE per episode  :  38.53
GFLOPs per episode   : 44.76
GFLOPs for 80 eps : 3580.92 



In [42]:
from agent.Ours_agent import MacLight
import torch

ckpt = torch.load(
    "ckpt/block_normal/Ours/42_Ours_win32.pt",
    map_location='cpu',
    weights_only=False)       # legacy behaviour
print(ckpt['agent']['B1'].actor.parameters())
  

<generator object Module.parameters at 0x000001CF5D0C6F80>
